# Praca domowa 4
### Sebastian Deręgowski

In [1]:
import warnings
warnings.filterwarnings('ignore')
import dalex as dx
import pandas as pd
import numpy as np
from dalex.datasets import load_apartments

Przyjrzyjmy się bliżej naszemu pierwszemu zbiorowi:

In [2]:
apartments = load_apartments()
apartments.head()

,m2_price,construction_year,surface,floor,no_rooms,district
1,5897,1953,25,3,1,Srodmiescie
2,1818,1992,143,9,5,Bielany
3,3643,1937,56,1,2,Praga
4,3517,1995,93,7,3,Ochota
5,3013,1992,144,6,5,Mokotow


In [3]:
apartments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 1 to 1000
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   m2_price           1000 non-null   int64 
 1   construction_year  1000 non-null   int64 
 2   surface            1000 non-null   int64 
 3   floor              1000 non-null   int64 
 4   no_rooms           1000 non-null   int64 
 5   district           1000 non-null   object
dtypes: int64(5), object(1)
memory usage: 54.7+ KB


Wszystkie wartości są nie-nullowe, jedna zmienna jest kategoryczna. Sprawdźmy ile różnych wartości przyjmuje:

In [4]:
apartments.district.value_counts()

Mokotow        107
Wola           106
Ursus          105
Ursynow        103
Srodmiescie    100
Bemowo          98
Zoliborz        97
Ochota          96
Bielany         96
Praga           92
Name: district, dtype: int64

Jest ich 10 i liczebność każdej z nich jest podobna, więc opłaca się zrobić OneHotEncoding.

In [5]:
apartments = pd.get_dummies(apartments,['district'])
apartments.head()

,m2_price,construction_year,surface,floor,no_rooms,district_Bemowo,district_Bielany,district_Mokotow,district_Ochota,district_Praga,district_Srodmiescie,district_Ursus,district_Ursynow,district_Wola,district_Zoliborz
1,5897,1953,25,3,1,0,0,0,0,0,1,0,0,0,0
2,1818,1992,143,9,5,0,1,0,0,0,0,0,0,0,0
3,3643,1937,56,1,2,0,0,0,0,1,0,0,0,0,0
4,3517,1995,93,7,3,0,0,0,1,0,0,0,0,0,0
5,3013,1992,144,6,5,0,0,1,0,0,0,0,0,0,0


Dokonajmy teraz podziału na zbiór treningowy i testowy:

In [6]:
from sklearn.model_selection import train_test_split
Xa_train, Xa_test, ya_train, ya_test = train_test_split(
    apartments.drop('m2_price',axis=1), apartments['m2_price'], test_size=0.25, random_state=42)

Jako drugi zbiór wybrałem zbiór danych nt. nieruchomości w Kalifornii z roku 1990, gdzie również zmienną celu jest wartość, tym razem całego mieszkania, a nie m^2.

In [7]:
housing = pd.read_csv("housing.csv")
housing.head()

,longitude,latitude,housing_median_age,total_bedrooms,population,households,median_income,median_house_value,<1H OCEAN,INLAND,NEAR BAY,NEAR OCEAN,rooms_per_household
0,-122.23,37.88,41.0,129.0,322.0,126.0,8.3252,452600.0,0.0,0.0,1.0,0.0,6.984127
1,-122.22,37.86,21.0,1106.0,2401.0,1138.0,8.3014,358500.0,0.0,0.0,1.0,0.0,6.238137
2,-122.24,37.85,52.0,190.0,496.0,177.0,7.2574,352100.0,0.0,0.0,1.0,0.0,8.288136
3,-122.25,37.85,52.0,235.0,558.0,219.0,5.6431,341300.0,0.0,0.0,1.0,0.0,5.817352
4,-122.25,37.85,52.0,280.0,565.0,259.0,3.8462,342200.0,0.0,0.0,1.0,0.0,6.281853


Zajmuję się tym zbiorem danych na zajęciach z Warsztatów Badawczych, więc powyższa ramka danych jest już po preprocessingu (m.in. dokonano OneHotEncodingu i usunięto kilka outlierów).

In [8]:
Xh_train, Xh_test, yh_train, yh_test = train_test_split(
    housing.drop('median_house_value',axis=1), housing['median_house_value'], test_size=0.25, random_state=42)

Pora na dopasowanie Support Vector Machine (w tym przypadku SVR, ponieważ jest to zadanie regresji):

In [9]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error as rmse
apartments_svr = SVR()
apartments_svr.fit(Xa_train,ya_train)

housing_svr = SVR()
housing_svr.fit(Xh_train,yh_train)

ya_hat = apartments_svr.predict(Xa_test)
yh_hat = housing_svr.predict(Xh_test)

print("Apartments:")
print(f"MSE (test): {rmse(ya_test,ya_hat,squared=False)}")
print(f"MSE (train): {rmse(ya_train,apartments_svr.predict(Xa_train),squared=False)}")
print()
print("Housing:")
print(f"MSE (test): {rmse(yh_test,yh_hat,squared=False)}")
print(f"MSE (train): {rmse(yh_train,housing_svr.predict(Xh_train),squared=False)}")

Apartments:
MSE (test): 917.6207825964947
MSE (train): 911.0643578066882

Housing:
MSE (test): 98135.813719618
MSE (train): 98958.94331600117


Co ciekawe, dla zbioru `housing` MSE na zbiorze testowym jest mniejsze niż na zbiorze treningowym.

Teraz przekonajmy się jak skalowanie zmiennych wpłynie na wyniki. W tym celu zastosujmy MinMaxScaler dla wszystkich zmiennych poza zmienną celu w obu ramkach danych.

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

apartments_dropped = apartments.drop('m2_price', axis = 1)
apartments_columns = apartments_dropped.columns
apartments_dropped = scaler.fit_transform(apartments_dropped)
apartments_dropped = pd.DataFrame(apartments_dropped, columns = apartments_columns).reset_index()
apartments_m2price = apartments['m2_price'].reset_index()
apartments_m2price = apartments_m2price.drop('index',axis=1)
apartments_scaled = pd.concat([apartments_dropped, apartments_m2price], axis = 1)
apartments_scaled = apartments_scaled.drop('index',axis=1)

scaler = MinMaxScaler()

housing_dropped = housing.drop('median_house_value', axis = 1)
housing_columns = housing_dropped.columns
housing_dropped = scaler.fit_transform(housing_dropped)
housing_dropped = pd.DataFrame(housing_dropped, columns = housing_columns)
housing_scaled = pd.concat([housing_dropped, housing['median_house_value']], axis = 1)

W przypadku ramki `apartments` psuło nam się indeksowanie (raz zaczynało od zera, raz od jedynki), więc należało to poprawić.

Teraz powtórzmy dzielenie na zbiór testowy i treningowy i obliczmy MSE:

In [11]:
Xas_train, Xas_test, yas_train, yas_test = train_test_split(
    apartments_scaled.drop('m2_price',axis=1), apartments_scaled['m2_price'], test_size=0.25, random_state=42)

Xhs_train, Xhs_test, yhs_train, yhs_test = train_test_split(
    housing_scaled.drop('median_house_value',axis=1), housing_scaled['median_house_value'], test_size=0.25, random_state=42)

In [12]:
apartments_scaled_svr = SVR()
apartments_scaled_svr.fit(Xas_train,yas_train)

housing_scaled_svr = SVR()
housing_scaled_svr.fit(Xhs_train,yhs_train)

yas_hat = apartments_scaled_svr.predict(Xas_test)
yhs_hat = housing_scaled_svr.predict(Xhs_test)

print("Apartments scaled:")
print(f"MSE (test): {rmse(yas_test,yas_hat,squared=False)}")
print(f"MSE (train): {rmse(yas_train,apartments_scaled_svr.predict(Xas_train),squared=False)}")
print()
print("Housing scaled:")
print(f"MSE (test): {rmse(yhs_test,yhs_hat,squared=False)}")
print(f"MSE (train): {rmse(yhs_train,housing_scaled_svr.predict(Xhs_train),squared=False)}")

Apartments scaled:
MSE (test): 894.3203586358629
MSE (train): 887.7116567070697

Housing scaled:
MSE (test): 97473.19169810312
MSE (train): 98326.84316353663


W przypadku obu ramek danych MSE zarówno zbiorów testowych, jak i treningowych zmniejszyły się, co potwierdza, że opłaca się dokonywać skalowania.

Na sam koniec zajmijmy się strojeniem parametrów `cost`, `gamma` i `degree`. Wykonamy go jedynie dla zbioru `apartments`, ponieważ zbiór `housing` jest o wiele większy (1000 obserwacji a blisko 20 tysięcy), a co za tym idzie strojenie zajmuje o wiele więcej czasu (po 40 minutach brak rezultatów).

In [13]:
from sklearn.model_selection import RandomizedSearchCV
params = {
         'C' : np.arange(0.01,10000.01,10),
         'gamma' : ['scale', 'auto'],
         'degree' : np.arange(1,10,2),
         }
apartments_rscv = RandomizedSearchCV(estimator=SVR(), param_distributions=params, n_iter= 100, n_jobs=-1)
apartments_rscv.fit(Xas_train, yas_train)
print(f"Apartments best parameters: {apartments_rscv.best_params_}")

Apartments best parameters: {'gamma': 'scale', 'degree': 3, 'C': 2520.01}


Spróbujmy zastosować powyższe parametry również dla zbioru `housing`. Sprawdźmy czy (a jeśli tak, to o ile) polepszyło się nasze MSE w obu przypadkach:

In [15]:
apartments_best = apartments_rscv.best_estimator_
housing_best = SVR(gamma='scale',degree=3,C=2520.01)
housing_best.fit(Xhs_train,yhs_train)

print("Apartments scaled+hipertuned:")
print(f"MSE (test): {rmse(yas_test,apartments_best.predict(Xas_test),squared=False)}")
print(f"MSE (train): {rmse(yas_train,apartments_best.predict(Xas_train),squared=False)}")
print()
print("Housing scaled+hipertuned:")
print(f"MSE (test): {rmse(yhs_test,housing_best.predict(Xhs_test),squared=False)}")
print(f"MSE (train): {rmse(yhs_train,housing_best.predict(Xhs_train),squared=False)}")

Apartments scaled+hipertuned:
MSE (test): 156.5535367279769
MSE (train): 133.85098275157912

Housing scaled+hipertuned:
MSE (test): 64735.9556997163
MSE (train): 65281.487469411964


Jak widzimy we wszystkich MSE jest najmniejsze spośród wszystkich dotychczasowych prób (i to znacznie), co podkreśla zarówno wagę strojenia parametrów, jak i skalowania.